In [ ]:
#!keithwhite@Keiths-MacBook-Pro/opt/anaconda3/envs/pyWAXS
# --------
# keith white
# date: 05/17/2023
# script: waxssim_main.py
# --
# purpose: simulate giwaxs data from inputted poscar file. import real data to compare
# to simulated data. select regions of interest in the real data to compare to the 
# simulated data. set a number of initial guesses for to the crystallite 
# orientation parameters. run a chi-square minimization routine to fit the 
# simulated data to the real data.
# --------

# -- functions -- #
# -- simulated data
# load poscar file
# read-in poscar file
# input parameters
    # crystallite orientation parameters
    # extent of (h k l) 
    # image resolution
    # scherrer grain-size analysis
# generate intensity map

# -- real data
# read-in .tiff file image
# create a detector corrections object for .tiff file image corrections
# apply detector corrections to the .tiff file image
# select regions of interest on the .tiff file image (2 - 4)
# find peaks on the tiff image file

# -- comparator functions
# create a project folder that includes CIFs, data, PONI, and corrected image .csv files
# check found peaks against simulated peak positions - group sets with same delta between
# reflectons
# check (qxy, qz) positions in poscar file against selected peaks in real data set
# modify lattice constants (a, b, c, alpha, beta, gamma) to match simulation to the real data
# set initial conditions for least squares minimization
# modify all gaussian parameters over specified step size in sigma, creating
# a 3D parameter space of textured images. 
# Use a peak finding algorithm to find the local minima of each mapped parameter space.
# Compare all local minima to find the global minima
# Output the lattice constants and correction parameters
# Output a list of the found peaks in the data, and whether they match the CIF or not.

In [ ]:
import numpy as np
from scipy import ndimage as ndi
from scipy.special import voigt_profile
from skimage import io, filters, feature
from skimage.feature import peak_local_max

def gaussian_noise(image, mean=0, std=1):
    return image + np.random.normal(mean, std, image.shape)

def generate_image(height, width, num_peaks, max_peak_width, max_peak_height):
    image = np.zeros((height, width))
    for _ in range(num_peaks):
        xc, yc = np.random.randint(0, height), np.random.randint(0, width)
        peak_width = np.random.randint(1, max_peak_width)
        peak_height = np.random.randint(1, max_peak_height)
        y, x = np.mgrid[:height, :width]
        image += peak_height * voigt_profile((x-xc)/peak_width, (y-yc)/peak_width, 0)
    return image

def find_peaks(image, min_distance=20, threshold_abs=None):
    smoothed_image = filters.gaussian(image, sigma=1)  # Adjust sigma according to your needs
    coordinates = peak_local_max(smoothed_image, min_distance=min_distance, threshold_abs=threshold_abs, exclude_border=True) 
    valid_peaks = []
    for coord in coordinates:
        x, y = coord
        if np.all(smoothed_image[max(0,x-1):min(image.shape[0],x+2), max(0,y-1):min(image.shape[1],y+2)] > 0):
            valid_peaks.append(coord)
    return np.array(valid_peaks)

def test_find_peaks():
    # Generate an image with random peaks and noise
    image = generate_image(200, 200, 10, 20, 50)
    image = gaussian_noise(image)
    peaks = find_peaks(image)
    print("Found peaks at: ", peaks)

test_find_peaks()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from scipy.special import voigt_profile
from skimage import io, filters, feature
from skimage.feature import peak_local_max

def gaussian_noise(image, mean=0, std=1):
    return image + np.random.normal(mean, std, image.shape)

def generate_image(height, width, num_peaks, max_peak_width, max_peak_height):
    image = np.zeros((height, width))
    for _ in range(num_peaks):
        xc, yc = np.random.randint(0, height), np.random.randint(0, width)
        peak_width = np.random.randint(1, max_peak_width)
        peak_height = np.random.randint(1, max_peak_height)
        y, x = np.mgrid[:height, :width]
        image += peak_height * voigt_profile((x-xc)/peak_width, (y-yc)/peak_width, 0)
    return image

def find_peaks(image, min_distance=20, threshold_abs=None):
    smoothed_image = filters.gaussian(image, sigma=1)  # Adjust sigma according to your needs
    coordinates = peak_local_max(smoothed_image, min_distance=min_distance, threshold_abs=threshold_abs, exclude_border=True) 
    valid_peaks = []
    for coord in coordinates:
        x, y = coord
        if np.all(smoothed_image[max(0,x-1):min(image.shape[0],x+2), max(0,y-1):min(image.shape[1],y+2)] > 0):
            valid_peaks.append(coord)
    return np.array(valid_peaks)

def visualize_image_and_peaks(image, peaks):
    plt.imshow(image)
    plt.scatter(peaks[:, 1], peaks[:, 0], color='red')  # Note the order of the coordinates
    plt.show()

def test_find_peaks():
    # Generate an image with random peaks and noise
    image = generate_image(200, 200, 10, 20, 50)
    image = gaussian_noise(image)
    
    # Find and visualize the peaks
    peaks = find_peaks(image)
    print("Found peaks at: ", peaks)
    visualize_image_and_peaks(image, peaks)

test_find_peaks()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from scipy.special import voigt_profile
from skimage import io, filters, feature
from skimage.feature import peak_local_max
%matplotlib widget

def gaussian_noise(image, mean=0, std=1):
    return image + np.random.normal(mean, std, image.shape)

def generate_image(height, width, num_peaks, max_peak_width, max_peak_height):
    image = np.zeros((height, width))
    for _ in range(num_peaks):
        xc, yc = np.random.randint(0, height), np.random.randint(0, width)
        peak_width = np.random.randint(1, max_peak_width)
        peak_height = np.random.randint(1, max_peak_height)
        y, x = np.mgrid[:height, :width]
        image += peak_height * voigt_profile((x-xc)/peak_width, (y-yc)/peak_width, 0)
    return image

def find_peaks(image, min_distance=20, threshold_abs=None):
    smoothed_image = filters.gaussian(image, sigma=1)  # Adjust sigma according to your needs
    coordinates = peak_local_max(smoothed_image, min_distance=min_distance, threshold_abs=threshold_abs, exclude_border=True) 
    valid_peaks = []
    for coord in coordinates:
        x, y = coord
        if np.all(smoothed_image[max(0,x-1):min(image.shape[0],x+2), max(0,y-1):min(image.shape[1],y+2)] > 0):
            valid_peaks.append(coord)
    return np.array(valid_peaks)

def visualize_image_and_peaks(image, peaks):
    plt.imshow(image, cmap='turbo')#, norm=plt.Normalize(vmin=np.min(image), vmax=np.max(image)))
    plt.scatter(peaks[:, 1], peaks[:, 0], color='black')  # Note the order of the coordinates
    plt.show()

def test_find_peaks(height, width, num_peaks, max_peak_width, max_peak_height):
    # Generate an image with random peaks and noise
    image = generate_image(height, width, num_peaks, max_peak_width, max_peak_height)
    image = gaussian_noise(image)
    
    # Find and visualize the peaks
    peaks = find_peaks(image)
    print("Found peaks at: ", peaks)
    visualize_image_and_peaks(image, peaks)

test_find_peaks(200, 200, 10, 80, 200)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from scipy.stats import multivariate_normal

def gaussian_noise(image, mean=0, std=1):
    return image + np.random.normal(mean, std, image.shape)

def generate_image(height, width, num_peaks, max_peak_width, max_peak_height):
    image = np.zeros((height, width))
    for _ in range(num_peaks):
        xc, yc = np.random.randint(0, height), np.random.randint(0, width)
        peak_width = np.random.randint(1, max_peak_width)
        peak_height = np.random.randint(1, max_peak_height)
        x, y = np.meshgrid(np.linspace(0, width, width), np.linspace(0, height, height))
        pos = np.empty(x.shape + (2,))
        pos[:, :, 0] = x; pos[:, :, 1] = y
        rv = multivariate_normal([xc, yc], [[peak_width, 0], [0, peak_width]])
        image += peak_height * rv.pdf(pos)
    return image

def find_peaks(image, size=3):
    maxima = (image == ndi.maximum_filter(image, size=size))
    labels, num_labels = ndi.label(maxima)
    peak_coords = np.array(ndi.center_of_mass(image, labels, range(1, num_labels+1)))
    return peak_coords

def visualize_image_and_peaks(image, peaks):
    plt.imshow(image, cmap='turbo', norm=plt.Normalize(vmin=np.min(image), vmax=np.max(image)))
    plt.scatter(peaks[:, 1], peaks[:, 0], color='black')  # Note the order of the coordinates
    plt.show()

def test_find_peaks(height, width, num_peaks, max_peak_width, max_peak_height):
    # Generate an image with random peaks and noise
    image = generate_image(height, width, num_peaks, max_peak_width, max_peak_height)
    image = gaussian_noise(image)
    
    # Find and visualize the peaks
    peaks = find_peaks(image)
    print("Found peaks at: ", peaks)
    visualize_image_and_peaks(image, peaks)

test_find_peaks(200, 200, 10, 20, 50)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from scipy.ndimage import gaussian_filter

%matplotlib widget

def gaussian_noise(image, mean=0, std=.1):
    return image + np.random.normal(mean, std, image.shape)

def random_gaussians(size=1000, num_gaussians=5, max_intensity=1):
    # Create a grid of coordinates for the array
    x, y = np.meshgrid(np.arange(size), np.arange(size))

    # Add multiple random Gaussian distributions to the array
    array = np.zeros((size, size))
    for _ in range(num_gaussians):
        x0, y0 = size * np.random.rand(2)
        x_sigma, y_sigma = size * (0.05 + 0.15*np.random.rand(2))
        amplitude = max_intensity / num_gaussians
        array += amplitude * np.exp(-((x-x0)**2/(2*x_sigma**2) + (y-y0)**2/(2*y_sigma**2)))

    return array

def plot_gaussians(array, points=None):
    plt.close('all')
    plt.imshow(array, cmap='turbo')
    plt.colorbar()
    if points is not None:
        plt.plot(points[:, 1], points[:, 0], 'ko', markersize=2)
    plt.show()

def find_peaks(image, size=3, smoothing_sigma=1, relative_threshold=1.5):
    # Smooth the image using a Gaussian filter
    smoothed = gaussian_filter(image, sigma=smoothing_sigma)
    
    # Find local maxima
    maxima = (smoothed == ndi.maximum_filter(smoothed, size=size))
    labels, num_labels = ndi.label(maxima)
    
    # Compute the noise level (standard deviation) in the image
    noise_level = np.std(image)
    
    # Determine a suitable absolute threshold based on the noise level
    absolute_threshold = relative_threshold * noise_level
    
    # Find the coordinates of the peaks that are above the threshold
    peak_coords = np.array(ndi.center_of_mass(image, labels, range(1, num_labels+1)))
    peak_values = smoothed[tuple(peak_coords.astype(int).T)]
    peak_coords = peak_coords[peak_values > absolute_threshold]
    
    return peak_coords

# Now the test function
def test_find_peaks(size, num_gaussians, max_intensity):
    # Generate an image with random peaks and noise
    image = random_gaussians(size, num_gaussians, max_intensity)
    image = gaussian_noise(image)
    
    # Find and visualize the peaks
    peaks = find_peaks(image)
    print("Found peaks at: ", peaks)
    plot_gaussians(image, peaks)
    
test_find_peaks(200, 10, 20)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from scipy.ndimage import gaussian_filter
from sklearn.cluster import DBSCAN
from skimage.restoration import estimate_sigma

%matplotlib widget

def gaussian_noise(image, mean=0, std=.1):
    return image + np.random.normal(mean, std, image.shape)

def random_gaussians(size=1000, num_gaussians=5, max_intensity=1):
    # Create a grid of coordinates for the array
    x, y = np.meshgrid(np.arange(size), np.arange(size))

    # Add multiple random Gaussian distributions to the array
    array = np.zeros((size, size))
    for _ in range(num_gaussians):
        x0, y0 = size * np.random.rand(2)
        x_sigma, y_sigma = size * (0.05 + 0.15*np.random.rand(2))
        amplitude = max_intensity / num_gaussians
        array += amplitude * np.exp(-((x-x0)**2/(2*x_sigma**2) + (y-y0)**2/(2*y_sigma**2)))

    return array

def plot_gaussians(array, points=None):
    plt.close('all')
    plt.imshow(array, cmap='turbo')
    plt.colorbar()
    if points is not None:
        plt.plot(points[:, 1], points[:, 0], 'ko', markersize=2)
    plt.show()

def cluster_peaks(peak_coords, eps=3, min_samples=2):
    # Apply DBSCAN to the peak coordinates
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(peak_coords)
    
    # Compute the centroid of each cluster
    clustered_peaks = []
    for cluster_id in np.unique(clustering.labels_):
        if cluster_id == -1:  # -1 means noise in DBSCAN
            continue
        coords_in_cluster = peak_coords[clustering.labels_ == cluster_id]
        centroid = np.mean(coords_in_cluster, axis=0)
        clustered_peaks.append(centroid)
        
    return np.array(clustered_peaks)

def calculate_snr(image):
    # Signal is the maximum pixel value
    signal = np.max(image)
    
    # Noise is estimated as the standard deviation of the pixel values
    noise = estimate_sigma(image, multichannel=False)
    
    # Calculate the SNR
    snr = signal / noise
    return snr

def find_peaks(image, size=3, smoothing_sigma=1, relative_threshold=None):
    # Smooth the image using a Gaussian filter
    smoothed = gaussian_filter(image, sigma=smoothing_sigma)
    
    # Find local maxima
    maxima = (smoothed == ndi.maximum_filter(smoothed, size=size))
    labels, num_labels = ndi.label(maxima)
    
    # Compute the noise level (standard deviation) in the image
    noise_level = estimate_sigma(image, multichannel=False)
    
    # If relative_threshold is None, set it based on the SNR
    if relative_threshold is None:
        snr = calculate_snr(image)
        relative_threshold = 1.5 if snr < 1 else 1/snr
    
    # Determine a suitable absolute threshold based on the noise level
    absolute_threshold = relative_threshold * noise_level
    
    # Find the coordinates of the peaks that are above the threshold
    peak_coords = np.array(ndi.center_of_mass(image, labels, range(1, num_labels+1)))
    peak_values = smoothed[tuple(peak_coords.astype(int).T)]
    peak_coords = peak_coords[peak_values > absolute_threshold]
    
    return peak_coords

def test_find_peaks(size, num_gaussians, max_intensity, size_peak=3, smoothing_sigma=1, relative_threshold=None, eps=3, min_samples=2):
    # Generate an image with random peaks and noise
    image = random_gaussians(size, num_gaussians, max_intensity)
    image = gaussian_noise(image)
    
    # Find and cluster the peaks
    peaks = find_peaks(image, size_peak, smoothing_sigma, relative_threshold)
    clustered_peaks = cluster_peaks(peaks, eps, min_samples)
    
    print("Found peaks at: ", clustered_peaks)
    plot_gaussians(image, clustered_peaks)

test_find_peaks(200, 10, 1)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from scipy.ndimage import gaussian_filter
from sklearn.cluster import DBSCAN
from skimage.restoration import estimate_sigma
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
%matplotlib widget

def gaussian_noise(image, mean=0, std=.1):
    return image + np.random.normal(mean, std, image.shape)

def random_gaussians(size=1000, num_gaussians=5, max_intensity=1):
    # Create a grid of coordinates for the array
    x, y = np.meshgrid(np.arange(size), np.arange(size))

    # Add multiple random Gaussian distributions to the array
    array = np.zeros((size, size))
    for _ in range(num_gaussians):
        x0, y0 = size * np.random.rand(2)
        x_sigma, y_sigma = size * (0.05 + 0.15*np.random.rand(2))
        amplitude = max_intensity / num_gaussians
        array += amplitude * np.exp(-((x-x0)**2/(2*x_sigma**2) + (y-y0)**2/(2*y_sigma**2)))

    return array

def plot_gaussians(array, points=None):
    plt.close('all')
    plt.imshow(array, cmap='turbo')
    plt.colorbar()
    if points is not None:
        plt.plot(points[:, 1], points[:, 0], 'ko', markersize=2)
    plt.show()

def cluster_peaks(peak_coords, eps=3, min_samples=2):
    # Apply DBSCAN to the peak coordinates
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(peak_coords)
    
    # Compute the centroid of each cluster
    clustered_peaks = []
    for cluster_id in np.unique(clustering.labels_):
        if cluster_id == -1:  # -1 means noise in DBSCAN
            continue
        coords_in_cluster = peak_coords[clustering.labels_ == cluster_id]
        centroid = np.mean(coords_in_cluster, axis=0)
        clustered_peaks.append(centroid)
        
    return np.array(clustered_peaks)

def calculate_snr(image):
    # Signal is the maximum pixel value
    signal = np.max(image)
    
    # Noise is estimated as the standard deviation of the pixel values
    noise = estimate_sigma(image, multichannel=False)
    
    # Calculate the SNR
    snr = signal / noise
    return snr

def find_peaks(image, size=3, smoothing_sigma=1, relative_threshold=None):
    # Smooth the image using a Gaussian filter
    smoothed = gaussian_filter(image, sigma=smoothing_sigma)
    
    # Find local maxima
    maxima = (smoothed == ndi.maximum_filter(smoothed, size=size))
    labels, num_labels = ndi.label(maxima)
    
    # Compute the noise level (standard deviation) in the image
    noise_level = estimate_sigma(image, multichannel=False)
    
    # If relative_threshold is None, set it based on the SNR
    if relative_threshold is None:
        snr = calculate_snr(image)
        relative_threshold = 1.5 if snr < 1 else 1/snr
    
    # Determine a suitable absolute threshold based on the noise level
    absolute_threshold = relative_threshold * noise_level
    
    # Find the coordinates of the peaks that are above the threshold
    peak_coords = np.array(ndi.center_of_mass(image, labels, range(1, num_labels+1)))
    peak_values = smoothed[tuple(peak_coords.astype(int).T)]
    peak_coords = peak_coords[peak_values > absolute_threshold]
    
    return peak_coords

def test_find_peaks(size, num_gaussians, max_intensity, size_peak=3, smoothing_sigma=1, relative_threshold=None, eps=3, min_samples=2):
    # Generate an image with random peaks and noise
    image = random_gaussians(size, num_gaussians, max_intensity)
    image = gaussian_noise(image)
    
    # Find and cluster the peaks
    peaks = find_peaks(image, size_peak, smoothing_sigma, relative_threshold)
    clustered_peaks = cluster_peaks(peaks, eps, min_samples)
    
    print("Found peaks at: ", clustered_peaks)
    plot_gaussians(image, clustered_peaks)

# Define a custom Dataset class for your images and peaks
class ImagePeakDataset(Dataset):
    def __init__(self, images, peaks):
        self.images = images
        self.peaks = peaks
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize to [-1, 1]
        ])
        
    def __len__(self):
        return len(self.images)
        
    def __getitem__(self, idx):
        image = self.transform(self.images[idx])
        peak = torch.tensor(self.peaks[idx], dtype=torch.float32)
        return image, peak

# Define a simple CNN architecture
class PeakFinderCNN(nn.Module):
    def __init__(self):
        super(PeakFinderCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc = nn.Linear(64, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Create your dataset and data loader
dataset = ImagePeakDataset(images, peaks)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Initialize your model, criterion and optimizer
model = PeakFinderCNN()
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(num_epochs):
    for i, (images, peaks) in enumerate(data_loader):
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, peaks)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
